In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from os.path import basename
import os


%matplotlib inline

#Function to fix the time (string)
def fix_time(df):
    '''This function takes the dataframe as an argument and 
    converts the string Lap Time column to be a datetime
    and then converts the datetime column to be a float.
    The function then returns the dataframe with those two
    columns appended to the right'''
    
    # TODO: Check to make sure that the 'Lap Time' column
    #       exists.
    # TODO: Check to make sure that the 'Lap Time' column
    #       is the right format
    
    df = df.assign(dtLapTime = pd.to_datetime('0:0:' + df['Lap Time'] + '0',
                                             exact = False, errors = 'ignore',
                                             format = "%H:%M:%S.%f").apply(lambda x: x.time()))
    df = df.assign(fLapTime = df.dtLapTime.apply(lambda x: ((x.hour * 60 + x.minute) * 60 + x.second) * 1000000 + x.microsecond) / 1000000)
    return df


def get_car_data(df, car_num, max_time):
    # make sure car exists
    # make a default max time val
    return df[(df['Car'] == car_num) & (df['Flag'] == 'Green') & (df['fLapTime'] < max_time)]


def get_cars_data(df, car_nums, max_time, strDfName):
    # make sure car exists
    # make a default max time val
    #df[df['A'].isin([3, 6])]
    df = df[(df['Car'].isin(car_nums)) & (df['Flag'] == 'Green') & (df['fLapTime'] < max_time)].copy()
    df['SetName'] = strDfName
    return df

def get_file_root(s_file_name):
    """trim off the file extension from the end of the path and trim off the path from the beginning of the path"""
    
    # print(os.path.splitext("path_to_file")[0])
    # now you can call it directly with basename
    #print(basename("/a/b/c.txt"))

    s_file_root = basename(os.path.splitext(s_file_name)[0]).replace(" ", "") #s_file_name.rtrim(3)
    
    return s_file_root

In [32]:
# Set Paths for Sebring 2018 data

s_path_root = 'C:\\Users\\gator\\Documents\\GitHub\\IMSA2017\\LapData\\2018_sebring\\'

s_path_gt3_p1 = s_path_root + 'GT3 Sebring Practice 1\\GT3 Sebring Practice 1.csv'
s_path_gt3_q = s_path_root + 'GT3 Sebring Quailfying\\GT3 Sebring Quailfying.csv'
s_path_gt3_r1 = s_path_root + 'GT3 Sebring Race 1 Replay\\GT3 Sebring Race 1.csv'
# TODO: Still need r2

s_path_pc_p1 = s_path_root + 'IPC Sebring Practice 1\\IPC Sebring Practice 1.csv'
s_path_pc_p2 = s_path_root + 'IPC Sebring Practice 2\\IPC Sebring Practice 2.csv'
s_path_pc_q = s_path_root + 'IPC Sebring Qualifying\\IPC Sebring Qualifying.csv'
# TODO: Still need r

s_path_ctsc_p1 = s_path_root + 'ICTSC Sebring Practice 1\\ICTSC Sebring Practice 1.csv'
s_path_ctsc_p2 = s_path_root + 'ICTSC Sebring Practice 2\\ICTSC Sebring Practice 2.csv'
s_path_ctsc_p3 = s_path_root + 'ICTSC Sebring Practice 3\\ICTSC Sebring Practice 3.csv'
s_path_ctsc_q = s_path_root + 'ICTSC Sebring Qualifying\\ICTSC Sebring Qualifying.csv'
# TODO: Still need r

s_path_wsc_p1 = s_path_root + 'IWSC Sebring Practice 1\\IWSC Sebring Practice 1.csv'
s_path_wsc_p2 = s_path_root + 'IWSC Sebring Practice 2\\IWSC Sebring Practice 2.csv'
s_path_wsc_p3 = s_path_root + 'IWSC Sebring Practice 3\\IWSC Sebring Practice 3.csv'
s_path_wsc_p4 = s_path_root + 'IWSC Sebring Practice 4\\IWSC Sebring Practice 4.csv'
# s_path_wsc_q = s_path_root + 'IWSC Sebring Qualifying//IWSC Sebring Qualifying.csv'
# s_path_wsc_r = s_path_root + 'IWSC Sebring Race//IWSC Sebring Race.csv'
# TODO: Still need q and r

In [33]:
# read in all the data from Sebring 2018

list_of_csv_files = [s_path_gt3_p1, s_path_gt3_q, s_path_gt3_r1,
                    s_path_pc_p1, s_path_pc_p2, s_path_pc_q, 
                    s_path_ctsc_p1, s_path_ctsc_p2, s_path_ctsc_p3, s_path_ctsc_q,
                    s_path_wsc_p1, s_path_wsc_p2, s_path_wsc_p3, s_path_wsc_p4]
df_list = []
for s_file_name in list_of_csv_files:
    df_temp = pd.read_csv(s_file_name)
    df_temp['Source'] = get_file_root(s_file_name)
    df_list.append(df_temp)
df = pd.concat(df_list)

In [34]:
df

,Car,Class,Driver,Lap,Lap Time,Session Time,Flag,Location,S01,S02,...,S04,S05,S06,S07,S08,S09,S10,SP5,SPI,Source
0,8,GT3CP,Stefan Rzadzinski,1,2:29.329,2:29.329,Green,Track,0.000,14.123,...,17.814,30.627,31.006,0.565,12.684,1.551,2.842,150.040,111.365,GT3SebringPractice1
1,56,GT3P,David Baker,1,2:30.168,2:30.168,Green,Track,227.993,14.299,...,17.390,30.602,31.424,0.603,12.770,1.546,2.833,140.585,111.725,GT3SebringPractice1
2,42,GT3G,Bill Smith,1,2:31.231,2:31.231,Green,Track,226.938,13.697,...,17.563,30.123,31.506,0.616,13.119,1.612,2.947,137.618,107.151,GT3SebringPractice1
3,10,GT3G,Frank Raso,1,2:32.579,2:32.579,Green,Track,227.083,11.293,...,17.346,29.517,32.463,0.628,12.917,1.614,2.954,134.988,107.018,GT3SebringPractice1
4,3,GT3P,Trenton Estep,1,2:33.854,2:33.854,Green,Track,5.948,0.000,...,0.000,0.000,0.000,0.000,0.000,1.552,2.835,0.000,111.293,GT3SebringPractice1
5,7,GT3P,Maxwell Root,1,2:38.312,2:38.312,Green,Track,220.561,10.907,...,16.849,29.718,30.330,0.548,12.409,1.521,2.800,154.695,113.562,GT3SebringPractice1
6,26,GT3G,Egidio Perfetti,1,2:39.029,2:39.029,Green,Track,220.342,11.381,...,17.698,29.801,31.416,0.605,12.189,1.560,2.871,140.120,110.723,GT3SebringPractice1
7,88,GT3CP,Marco Cirone,1,2:43.806,2:43.806,Green,Track,222.339,11.610,...,17.382,32.957,33.591,0.606,12.426,1.540,2.823,139.889,112.161,GT3SebringPractice1
8,34,GT3P,Shaun McKaigue,1,2:50.299,2:50.299,Green,Track,132.856,12.267,...,17.593,30.374,30.185,0.556,12.521,1.576,2.861,152.469,109.599,GT3SebringPractice1
9,19,GT3P,Will Hardeman,1,2:47.138,2:47.138,Green,NaN,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,GT3SebringPractice1
